In [10]:
import joblib
from src import Utils, EnsembleFunctions, BERT
import pandas as pd
import numpy as np

/home/nate/miniconda3/lib/python3.9/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/nate/miniconda3/lib/python3.9/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
import torch
torch.cuda.empty_cache()
torch.cuda.is_available = lambda: False

In [3]:
Utils.seed_random_number_generators()

Random number generators seeded.


In [11]:
BertModel = BERT.BertModel

def load_model(model_name: str):
  try:
    return joblib.load(model_name)
  except FileNotFoundError:
    print("ERROR: Model not found")
    return None

learner_bayes = load_model('model_bayes/Bayes.pkl')
learner_lstm = load_model('model_lstm/LSTM.pkl')
learner_bert = load_model('model_bert/mBERT.pkl')
learner_lr = load_model('model_lr/LR.pkl')

/home/nate/miniconda3/lib/python3.9/site-packages/skorch/utils.py:568: DeviceWarning: Requested to load data to CUDA but no CUDA devices are available. Loading on device "cpu" instead.
  warnings.warn(
/home/nate/miniconda3/lib/python3.9/site-packages/skorch/net.py:2539: DeviceWarning: Setting self.device = cpu since the requested device (cuda) is not available.
  warnings.warn(
/home/nate/miniconda3/lib/python3.9/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.2.2 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/nate/miniconda3/lib/python3.9/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.3.1. This might lead to breaking c

In [5]:
learner_bayes

Pipeline(steps=[('tfidf', CountVectorizer()),
                ('bayes', BernoulliNB(alpha=0.3))])

In [6]:
learner_lstm

Pipeline(steps=[('tokenizer', CalamancyTokenizer()),
                ('lstm',
                 <class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=LstmModel(
    (lstm): LSTM(200, 250, batch_first=True)
    (fc): Linear(in_features=250, out_features=2, bias=True)
  ),
))])

In [12]:
learner_bert

Pipeline(steps=[('tokenizer',
                 HuggingfacePretrainedTokenizer(max_length=512,
                                                tokenizer='bert-base-multilingual-cased')),
                ('bert',
                 <class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=BertModel(
    (bert): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(119547, 768, padding_idx=0)
          (position_e...
                (dense): Linear(in_features=3072, out_features=768, bias=True)
                (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
            )
          )
        )
        (pooler): BertPooler(
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (activation): Tanh()
        )
      )
      (dropout): Dropout(p=0.1, inplace=False)
      (classifier): Linear(in_features=768, out_features=2, bias=True)
    )
  ),
))])

In [13]:
learner_lr

LogisticRegression(C=0.0001)

In [14]:
dataset = Utils.read_csv_file('datasets/datasetall.csv')

CSV file read successfully!


In [15]:
TEST_SIZE = 0.1

X_train, X_test, y_train, y_test = Utils.get_train_test_split(dataset, TEST_SIZE)

In [16]:
X_train

0        pag hindi nanalo si Norberto Gonzales pwede ba...
1        Ngayon lang ako super proud sa PRESIDENTE na i...
2        JUST SAW SOMEONE CALL BBM BLENGBLONG HAHAHAHAH...
3        Rep. Binay on her leadership style: I am very ...
4        Liwanag o dilim? May oras pa. Kakampink Leni L...
                               ...                        
25611    "Kala ko wala andito pala si Marcos."*pertaini...
25612    cathy [USERNAME] Dec [USERNAME] parang tanga i...
25613                             Nognog+pandak= BINAY ftw
25614    BINAY:Did your enormous wealth all come from y...
25615                                Uunlad tayo kay Binay
Name: text, Length: 25616, dtype: object

In [17]:
y_train

0        1
1        0
2        1
3        0
4        0
        ..
25611    0
25612    1
25613    1
25614    1
25615    0
Name: label, Length: 25616, dtype: int64

In [18]:
VAL_SPLIT = 0.5

X_val, X_test, y_val, y_test = Utils.get_train_test_split(
  pd.DataFrame({
    'text': X_test,
    'label': y_test,
  }), 
  VAL_SPLIT,
)

In [19]:
X_test

0       lexi miner [USERNAME] Dec [USERNAME] Leody De ...
1                                   Sooo not Mar Roxas ??
2       [USERNAME]and[USERNAME] Si VP LENI ay NAGAWA a...
3       ??Only Binayonly Binay?? Tama na Binay! Kakasi...
4       An honor to be part of this to know and to be ...
                              ...                        
1417    Pulse Asia: Duterte pulls away in presidential...
1418    In reality, we should let them die, is that wh...
1419    PDI reporter Carvajalwhose reports were quoted...
1420                Nababadtrip talaga ako sa ad ni binay
1421                                        Dapat Si Leni
Name: text, Length: 1422, dtype: object

In [20]:
y_test

0       0
1       0
2       0
3       1
4       0
       ..
1417    0
1418    0
1419    0
1420    1
1421    0
Name: label, Length: 1422, dtype: int64

In [ ]:
from src import EnsembleFunctions

predictions = EnsembleFunctions.concatenate_predict_proba(
  learner_bayes,
  learner_lstm,
  learner_bert,
  X_train,
)

predictions

KeyboardInterrupt: 

In [ ]:
soft_voting_results_discrete = np.argmax(soft_voting_results, axis=1)

soft_voting_results_discrete

array([1, 1, 0, ..., 1, 1, 0])

In [ ]:
pd.DataFrame([{
  'accuracy': accuracy_score(soft_voting_results_discrete, y_test),
  'recall': recall_score(soft_voting_results_discrete, y_test),
  'precision': precision_score(soft_voting_results_discrete, y_test),
  'f1': f1_score(soft_voting_results_discrete, y_test),
}])

,accuracy,recall,precision,f1
0,0.846397,0.821821,0.887727,0.853503


In [ ]:
stacking_results = EnsembleFunctions.stacking(
  learner_bayes,
  learner_lstm,
  learner_bert,
  learner_lr,
  X_test
)

stacking_results

array([[0.0935605 , 0.9064395 ],
       [0.07668313, 0.92331687],
       [0.98238279, 0.01761721],
       ...,
       [0.05485927, 0.94514073],
       [0.05103099, 0.94896901],
       [0.96259484, 0.03740516]])

In [ ]:
stacking_results_discrete = np.argmax(stacking_results, axis=1)

stacking_results_discrete

array([1, 1, 0, ..., 1, 1, 0])

In [ ]:
pd.DataFrame([{
  'accuracy': accuracy_score(stacking_results_discrete, y_test),
  'recall': recall_score(stacking_results_discrete, y_test),
  'precision': precision_score(stacking_results_discrete, y_test),
  'f1': f1_score(stacking_results_discrete, y_test),
}])

,accuracy,recall,precision,f1
0,0.849912,0.846524,0.857741,0.852096
